In [ ]:
import cv2
from cv2 import VideoCapture
import numpy as np
from tqdm import tqdm
from typing import Callable

In [ ]:
def calcBackground(
    video_path: str,
    num_samples: int = 100,
    image_transform: Callable[[np.ndarray], np.ndarray] = None,
) -> np.ndarray:
    # Open Video
    cap = VideoCapture(video_path)

    # Get Video Length
    length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Randomly select up to 50 frames
    frame_ids = length * np.random.uniform(size=min(length, num_samples))

    # Store selected frames in an array
    frames = []
    for fid in tqdm(frame_ids, desc="reading frames"):
        cap.set(cv2.CAP_PROP_POS_FRAMES, fid)
        ret, frame = cap.read()

        # Apply transform if needed
        if image_transform is not None:
            frame = image_transform(frame)

        # Convert to grayscale
        frames.append(frame)

    # Calculate the median along the time axis
    median_frame = np.median(frames, axis=0).astype(dtype=np.uint8)

    return median_frame

In [ ]:
def toGrayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

In [ ]:
def extractBoxes(
    image: np.ndarray,
    background: np.ndarray,
    diff_thresh: int = 75,
    size_thresh: int = 40,
) -> np.ndarray:
    # Calculate difference between background and image
    diff = np.abs(image.astype(np.int32) - background.astype(np.int32)).astype(np.uint8)

    # Turn differences mask to black & white according to a threshold value
    _, mask = cv2.threshold(diff, diff_thresh, 255, cv2.THRESH_BINARY)

    # find contours in the binary mask
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Populate bounding boxes
    bboxes = []
    for c in contours:
        rect = cv2.boundingRect(c)

        # Skip too small bounding boxes
        if rect[2] < size_thresh and rect[3] < size_thresh:
            continue

        bboxes.append(rect)

    return bboxes

In [ ]:
def extractRois(image: np.ndarray, bboxes: list, width: int = 200, height: int = 200):
    rois = []
    for box in bboxes:
        x, y, w, h = box

        # Calculate center of bbox
        x_center, y_center = x + w // 2, y + h // 2

        # Get bottom left corner
        x0, y0 = x_center - width // 2, y_center - height // 2
        x0, y0 = max(0, x0), max(0, y0)

        roi = image[y0 : y0 + height, x0 : x0 + width]

        rois.append(roi)

    return rois

In [ ]:
def FindRois(
    video_path: str,
    video_background: np.ndarray,
    background_diff_thresh: int = 75,
    bbox_size_thresh: int = 40,
    image_transform: Callable[[np.ndarray], np.ndarray] = None,
):
    cap = VideoCapture(video_path)

    while True:
        ret, image = cap.read()
        if ret == False:
            break

        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

        if image_transform is not None:
            imgTrans = image_transform(image)

        bboxes = extractBoxes(imgTrans, background=video_background, diff_thresh=background_diff_thresh, size_thresh=bbox_size_thresh)

        rois = extractRois(image, bboxes, width=200, height=200)

        # Draw bboxes
        for box in bboxes:
            x, y, w, h = box

            cv2.rectangle(image, (x, y), (x + w, y + h), (0, 0, 255), 3)

        cv2.imshow("BBOXES", image)

    cap.release()
    cv2.destroyAllWindows()

In [ ]:
bg_gray = calcBackground("worms.avi", num_samples=100, image_transform=toGrayscale)

In [ ]:
FindRois("worms.avi", bg_gray, image_transform=toGrayscale)

In [ ]:
cv2.destroyAllWindows()